In [1]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [2]:
import os
os.chdir("/project/work/Passenger_Demand")

## 패키지 설치
!pip install seaborn
!pip install haversine
!pip install pyarrow
!pip install multiprocessing_on_dill
!pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
import math
import numbers

from haversine import haversine
import seaborn as sns
import matplotlib.pyplot as plt

In [54]:
import datetime

In [209]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수

        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key

        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
    #func = partial(func, **params)
    
    def map_func(data):
        return data.apply(func, axis = 1, group_data= data.copy() ,**params)
        
        
    with Pool(num_cores) as p:

        pd_result = pd.concat(p.map(map_func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [4]:
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'Nanum Gothic'
sns.set(font="NanumGothic")

In [14]:
%%time
bus_demand_401 = pd.read_parquet('model_mr/mybicard_401_agg.parquet', engine='pyarrow')

CPU times: user 40.9 ms, sys: 54 ms, total: 94.9 ms
Wall time: 26 ms


In [62]:
bus_demand_401.sort_values('transdate').head(10)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
223368,3101482,2020-04-08 00:00:00,1,1,0,2
16084,3100144,2020-04-08 00:00:00,2,0,0,2
89825,3101414,2020-04-08 00:00:00,0,1,0,1
164705,3101446,2020-04-08 00:00:00,1,0,0,1
156351,3101442,2020-04-08 00:00:00,1,0,0,1
95714,3101416,2020-04-08 00:00:00,1,0,0,1
150970,3101440,2020-04-08 00:00:00,1,0,0,1
278482,3101537,2020-04-08 05:00:00,2,0,0,2
293924,3101542,2020-04-08 05:00:00,16,0,0,16
13909,3100101,2020-04-08 05:00:00,1,0,0,1


In [57]:
bus_demand_401_check = bus_demand_401.copy()
bus_demand_401_check['date'] = bus_demand_401.apply(lambda x: x['transdate'].date(), axis=1)

In [61]:
bus_demand_401_check.groupby(['mybi_stop_id']).nunique()

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt,date
mybi_stop_id,,,,,,,
3100020,1,2797,15,6,1,16,190
3100021,1,3311,25,5,3,23,190
3100057,1,1518,19,3,2,19,190
3100058,1,2102,20,4,3,19,190
3100085,1,1150,6,6,2,8,189
3100086,1,3031,15,5,4,14,190
3100101,1,1444,8,4,1,8,190
3100106,1,37,4,2,1,3,35
3100142,1,315,7,21,1,22,140


In [119]:
#처음날짜, 마지막날짜
min_date, max_date = bus_demand_401['transdate'].min(), bus_demand_401['transdate'].max()
min_date, max_date

(Timestamp('2020-04-08 00:00:00'), Timestamp('2020-11-30 23:00:00'))

In [51]:
#전체 일수 계산
( bus_demand_401['transdate'].max() - bus_demand_401['transdate'].min() ).days

236

In [70]:
#전체 날짜 생성
entire_date = [min_date + datetime.timedelta(days= i) for i in range(237)]
entire_date[:1], entire_date[-1:]

([Timestamp('2020-04-08 00:00:00')], [Timestamp('2020-11-30 00:00:00')])

In [78]:
#보유데이터 시각 추출
hours = bus_demand_401.apply(lambda x: x['transdate'].hour, axis=1).unique()
hours.sort(); hours

array([ 0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23])

In [85]:
#전체 transdate 생성
transdate = [x + datetime.timedelta(hours= int(y)) for x in entire_date for y in hours]
transdate[18:22]

[Timestamp('2020-04-08 22:00:00'),
 Timestamp('2020-04-08 23:00:00'),
 Timestamp('2020-04-09 00:00:00'),
 Timestamp('2020-04-09 05:00:00')]

In [87]:
#보유데이터 정류장번호 추출
stop_ids = bus_demand_401['mybi_stop_id'].unique()
len(stop_ids)

131

In [113]:
data = {'mybi_stop_id' : [stop_id for stop_id in stop_ids for i in range(len(transdate))],
        'transdate' : transdate * len(stop_ids)}
bus_stop_df = pd.DataFrame(data= data)

In [115]:
bus_stop_df.head(3)

,mybi_stop_id,transdate
0,3100020,2020-04-08 00:00:00
1,3100020,2020-04-08 05:00:00
2,3100020,2020-04-08 06:00:00


In [195]:
bus_demand_401.head(3)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
0,3100020,2020-04-08 06:00:00,2,0,0,2
1,3100020,2020-04-08 08:00:00,7,0,0,7
2,3100020,2020-04-08 10:00:00,2,0,0,2


In [196]:
aa = pd.merge(bus_stop_df, bus_demand_401, on=['mybi_stop_id', 'transdate'], how='left')

In [207]:
aa.shape

(620940, 6)

In [281]:
def fill_blank_data(data, group_data):
    data_copy = data.copy()
    # 데이터 있는지 확인
    if math.isnan(data_copy['totalcnt']) : # 데이터 없음
        col_names = ['normalcnt','studentcnt','childcnt','totalcnt']
        subset_df = group_data.sort_values(['transdate']).reset_index(drop=True).copy()

        # 해당 일의 데이터가 있는지 확인
        check_day_df = subset_df[subset_df.apply(lambda x: x['transdate'].date(), axis=1) == data_copy['transdate'].date()][col_names].copy()
        
        if len(check_day_df) == 0 : # 해당 일의 데이터 없음
            
            # 4주 전까지의 데이터 확인
            date_4W_list = [data_copy['transdate'] - datetime.timedelta(weeks= i) for i in range(1,5)]
            if len(date_list) == 0 : # 4주 전까지의 데이터 없음
                tmp_date = data_copy['transdate'].date()
                
                # n주 전 데이터 찾기
                while tmp_date < subset_df['transdate'].min() :
                    result_data = subset_df[subset_df['transdate'] == tmp_date][col_names].copy()
                    if len(result_data) == 0 :
                        tmp_date -= datetime.timedelta(weeks= 1)
                    else :
                        break
                        
                # n주 전 데이터 찾았는지 확인
                if len(result_data) == 0: # False
                    result_data = pd.DataFrame([[data_copy['mybi_stop_id'], data_copy['transdate'], 0, 0, 0, 0]], columns= subset_df.columns)
                    
            else : # 4주 전까지의 데이터 있음
                result_data = subset_df[subset_df['transdate'].isin(date_4W_list)].groupby('transdate').mean() \
                            .apply(lambda x: math.ceil(x), axis=0).T[col_names].copy()
                
        else : # 해당 일의 데이터 있음
            result_data = pd.DataFrame([[data_copy['mybi_stop_id'], data_copy['transdate'], 0, 0, 0, 0]], columns= subset_df.columns)
            
    else : # 데이터 있음
        result_data = pd.DataFrame([data_copy])
    return result_data.reset_index(drop=True).loc[0]

In [289]:
%%time
bb = parallelize_dataframe(df = aa, func = fill_blank_data, group_keys = 'mybi_stop_id', num_cores = 40)

CPU times: user 42.5 s, sys: 1min 2s, total: 1min 45s
Wall time: 4h 47min 50s


In [290]:
bb.shape

(620940, 6)

In [313]:
bb.head(10)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
0,3100020,2020-04-08 00:00:00,0.0,0.0,0.0,0.0
1,3100020,2020-04-08 05:00:00,0.0,0.0,0.0,0.0
2,3100020,2020-04-08 06:00:00,2.0,0.0,0.0,2.0
3,3100020,2020-04-08 07:00:00,0.0,0.0,0.0,0.0
4,3100020,2020-04-08 08:00:00,7.0,0.0,0.0,7.0
5,3100020,2020-04-08 09:00:00,0.0,0.0,0.0,0.0
6,3100020,2020-04-08 10:00:00,2.0,0.0,0.0,2.0
7,3100020,2020-04-08 11:00:00,2.0,0.0,0.0,2.0
8,3100020,2020-04-08 12:00:00,1.0,0.0,0.0,1.0
9,3100020,2020-04-08 13:00:00,2.0,1.0,0.0,3.0


In [294]:
bb.isnull().sum()

mybi_stop_id    0
transdate       0
normalcnt       0
studentcnt      0
childcnt        0
totalcnt        0
dtype: int64

In [292]:
bb.to_parquet("model_mr/mybicard_401_agg_imputation.parquet")

In [316]:
def period_before_cnt(data, group_data, i):
    col_names = ['normalcnt','studentcnt','childcnt','totalcnt']
    data_copy = data.copy()
    subset_df = group_data.sort_values(['transdate']).reset_index(drop=True).copy()
    
    before_day = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(days=i))][col_names].copy()
    before_week = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(weeks=i))][col_names].copy()
    
    if len(before_day) == 0:
        before_day = pd.DataFrame([data_copy[col_names]])
    if len(before_week) == 0:
        before_week = pd.DataFrame([data_copy[col_names]])
    
    for name in col_names:
        data_copy[name + '_D-' + str(i)] = before_day.reset_index(drop=True).loc[0][name]
        data_copy[name + '_W-' + str(i)] = before_week.reset_index(drop=True).loc[0][name]
    return data_copy

In [319]:
%%time
cc = parallelize_dataframe(df = bb, func = period_before_cnt, group_keys = 'mybi_stop_id', num_cores = 10, i= 1)

CPU times: user 2.81 s, sys: 3.2 s, total: 6.01 s
Wall time: 41min 3s


In [320]:
cc.to_parquet("model_mr/mybicard_401_agg_add_1.parquet")

In [321]:
%%time
dd = parallelize_dataframe(df = cc, func = period_before_cnt, group_keys = 'mybi_stop_id', num_cores = 20, i= 3)

CPU times: user 7.48 s, sys: 9.43 s, total: 16.9 s
Wall time: 1h 11min 27s


In [323]:
dd.to_parquet("model_mr/mybicard_401_agg_add_3.parquet")

In [324]:
%%time
ee = parallelize_dataframe(df = dd, func = period_before_cnt, group_keys = 'mybi_stop_id', num_cores = 20, i= 4)

CPU times: user 8.04 s, sys: 9.84 s, total: 17.9 s
Wall time: 1h 11min 6s


In [325]:
%%time
ff = parallelize_dataframe(df = ee, func = period_before_cnt, group_keys = 'mybi_stop_id', num_cores = 8, i= 2)

CPU times: user 3.28 s, sys: 3.05 s, total: 6.33 s
Wall time: 38min 7s


In [326]:
ff.columns

Index(['mybi_stop_id', 'transdate', 'normalcnt', 'studentcnt', 'childcnt',
       'totalcnt', 'normalcnt_D-1', 'normalcnt_W-1', 'studentcnt_D-1',
       'studentcnt_W-1', 'childcnt_D-1', 'childcnt_W-1', 'totalcnt_D-1',
       'totalcnt_W-1', 'normalcnt_D-3', 'normalcnt_W-3', 'studentcnt_D-3',
       'studentcnt_W-3', 'childcnt_D-3', 'childcnt_W-3', 'totalcnt_D-3',
       'totalcnt_W-3', 'normalcnt_D-4', 'normalcnt_W-4', 'studentcnt_D-4',
       'studentcnt_W-4', 'childcnt_D-4', 'childcnt_W-4', 'totalcnt_D-4',
       'totalcnt_W-4', 'normalcnt_D-2', 'normalcnt_W-2', 'studentcnt_D-2',
       'studentcnt_W-2', 'childcnt_D-2', 'childcnt_W-2', 'totalcnt_D-2',
       'totalcnt_W-2'],
      dtype='object')

In [332]:
ff.shape

(620940, 38)

In [330]:
ff[col]

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt,normalcnt_D-1,studentcnt_D-1,childcnt_D-1,totalcnt_D-1,normalcnt_D-2,studentcnt_D-2,childcnt_D-2,totalcnt_D-2,normalcnt_D-3,studentcnt_D-3,childcnt_D-3,totalcnt_D-3,normalcnt_D-4,studentcnt_D-4,childcnt_D-4,totalcnt_D-4,normalcnt_W-1,studentcnt_W-1,childcnt_W-1,totalcnt_W-1,normalcnt_W-2,studentcnt_W-2,childcnt_W-2,totalcnt_W-2,normalcnt_W-3,studentcnt_W-3,childcnt_W-3,totalcnt_W-3,normalcnt_W-4,studentcnt_W-4,childcnt_W-4,totalcnt_W-4
0,3100020,2020-04-08 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3100020,2020-04-08 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3100020,2020-04-08 06:00:00,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0
3,3100020,2020-04-08 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3100020,2020-04-08 08:00:00,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0,7.0,0.0,0.0,7.0
5,3100020,2020-04-08 09:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,3100020,2020-04-08 10:00:00,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0
7,3100020,2020-04-08 11:00:00,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0
8,3100020,2020-04-08 12:00:00,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
9,3100020,2020-04-08 13:00:00,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0,2.0,1.0,0.0,3.0


In [329]:
col = ['mybi_stop_id', 'transdate', 'normalcnt', 'studentcnt', 'childcnt', 'totalcnt'
       , 'normalcnt_D-1', 'studentcnt_D-1', 'childcnt_D-1', 'totalcnt_D-1'
       , 'normalcnt_D-2', 'studentcnt_D-2', 'childcnt_D-2', 'totalcnt_D-2'
       , 'normalcnt_D-3', 'studentcnt_D-3', 'childcnt_D-3', 'totalcnt_D-3'
       , 'normalcnt_D-4', 'studentcnt_D-4', 'childcnt_D-4', 'totalcnt_D-4'
       , 'normalcnt_W-1', 'studentcnt_W-1', 'childcnt_W-1','totalcnt_W-1'
       , 'normalcnt_W-2', 'studentcnt_W-2', 'childcnt_W-2','totalcnt_W-2'
       , 'normalcnt_W-3', 'studentcnt_W-3', 'childcnt_W-3','totalcnt_W-3'
       , 'normalcnt_W-4', 'studentcnt_W-4', 'childcnt_W-4','totalcnt_W-4']

In [331]:
ff[col].to_parquet("model_mr/mybicard_401_agg_add_4.parquet")

In [ ]:
def moving_average_cnt(data, group_data, i):
    col_names = ['normalcnt','studentcnt','childcnt','totalcnt']
    data_copy = data.copy()
    subset_df = group_data.sort_values(['transdate']).reset_index(drop=True).copy()
    
    before_day = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(days=i))][col_names].copy()
    before_week = subset_df[(subset_df['transdate'] == data_copy['transdate'] - datetime.timedelta(weeks=i))][col_names].copy()
    
    if len(before_day) == 0:
        before_day = pd.DataFrame([data_copy[col_names]])
    if len(before_week) == 0:
        before_week = pd.DataFrame([data_copy[col_names]])
    
    for name in col_names:
        data_copy[name + '_D-' + str(i)] = before_day.reset_index(drop=True).loc[0][name]
        data_copy[name + '_W-' + str(i)] = before_week.reset_index(drop=True).loc[0][name]
    return data_copy